In [ ]:
# 2. Import libraries
import os
import time
import cupy as cp
import numpy as np # We still need numpy for CPU operations with Pandas
import pandas as pd
from cupyx.scipy.ndimage import binary_dilation

In [ ]:
# 3. Define paths and structure
base_dir = r'E:\Process'

# Check if the directory exists
if not os.path.exists(base_dir):
    print(f"The directory {base_dir} does not exist.")
else:
    print(f"Directory set: {base_dir}")

# Structure for binary dilation - stays on GPU
structure = cp.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]], dtype=cp.uint8)

Diretório definido: E:\Process


In [ ]:


# 4. Optimized processing functions
def analyze_neighborhood_gpu_vectorized(patches_A_gpu, patches_B_gpu):
    print("Starting neighborhood analysis on GPU (vectorized)...")
    start_time = time.time()

    # Get unique patch IDs in patches_A, ignoring 0
    patch_ids_A = cp.unique(patches_A_gpu[patches_A_gpu > 0])
    total_patches = len(patch_ids_A)
    print(f"Total unique patches to analyze: {total_patches}")

    # Initialize arrays to store results on the GPU
    results_patch_id = cp.empty(total_patches, dtype=cp.int32)
    results_vizinhanza = cp.empty(total_patches, dtype=cp.uint8)
   
    max_patch_id_A = int(patch_ids_A.max()) # Convert to int to avoid indexing issues
    # Map to store results
    vizinhanza_map = cp.zeros(max_patch_id_A + 1, dtype=cp.uint8)

    # Process in blocks to manage GPU memory and progress
    GPU_BLOCK_SIZE = 1000 # Adjust this value based on GPU memory and number of patches

    for i in range(0, total_patches, GPU_BLOCK_SIZE):
        block_ids = patch_ids_A[i : min(i + GPU_BLOCK_SIZE, total_patches)]
        print(f"  Processing patch block {i+1} to {min(i + GPU_BLOCK_SIZE, total_patches)} of {total_patches} on GPU...") 
    all_patch_ids_A = cp.unique(patches_A_gpu[patches_A_gpu > 0])
    total_unique_patches = len(all_patch_ids_A)

    # Prepare arrays for results, already on the GPU
    results_patch_ids = cp.empty(total_unique_patches, dtype=cp.int32)
    results_vizinhanza = cp.empty(total_unique_patches, dtype=cp.uint8)

       # This is the main optimization point for your case.
    for i, patch_id_val in enumerate(all_patch_ids_A):
        # The operations below are all CuPy and run on the GPU
        mask_A = patches_A_gpu == patch_id_val
        dilated = binary_dilation(mask_A, structure=structure)
        border = cp.logical_and(dilated, ~mask_A)

        neighbor_values = patches_B_gpu[border]
        neighbor_ids = cp.unique(neighbor_values[neighbor_values > 0])

        # Neighborhood logic (still serialized by patch_id, but running fast on the GPU) 
        viz = 0 # Initialize with a default value in case no condition is met
        if len(neighbor_ids) == 0:
            viz = 1
        elif len(neighbor_ids) == 1:
            # Note: cp.any and cp.all in CuPy operate on CuPy arrays.
            # If neighbor_values is a CuPy array, cp.any/cp.all will work.
            if cp.any(neighbor_values == 0): # Check if there is any "stable" pixel (value 0) in the neighborhood
                viz = 2
            elif cp.all(neighbor_values == neighbor_ids[0]): # Check if all neighbor pixels have the same ID
                viz = 4
            else: # Different IDs in the neighborhood, but only one unique neighbor_id (e.g., 0 and another ID)
                viz = 2
        else: # More than one neighbor_id
            viz = 3

        results_patch_ids[i] = patch_id_val
        results_vizinhanza[i] = viz

        if (i + 1) % 1000 == 0: # Progress feedback 
            print(f"  Processed {i+1} of {total_unique_patches} patches...")
            cp.cuda.Stream.null.synchronize() # Ensure GPU operations are completed for accurate timing

    # Transfer the final results from GPU to CPU ONLY ONCE
    final_patch_ids = cp.asnumpy(results_patch_ids)
    final_vizinhanza = cp.asnumpy(results_vizinhanza)

    print(f"Neighborhood analysis completed in {time.time() - start_time:.2f} seconds.")
    return list(zip(final_patch_ids, final_vizinhanza))


# 5. Check GPU (remains the same)
print("GPU available:", cp.cuda.Device(0).attributes.get('name', 'Name not available'))
cp.cuda.Device(0).use() # Ensure the default GPU is used

# 6. Load .npy files (remains the same, load directly to GPU)
print("Loading .npy files to GPU...")
ganho = cp.load(os.path.join(base_dir, 'ganho_patches_2013_2018.npy'))
perda = cp.load(os.path.join(base_dir, 'perda_patches_2013_2018.npy'))
estavel = cp.load(os.path.join(base_dir, 'estavel_patches_2013_2018.npy'))
print("Files loaded to GPU.")
# 7. Execute analysis with the optimized function
print("\nAnalyzing loss...")
resultado_perda = analisar_vizinhança_gpu_vetorizado(perda, estavel)

print("\nAnalyzing gain...")
resultado_ganho = analisar_vizinhança_gpu_vetorizado(ganho, estavel)


# 8. Save results to CSV (remains the same)
print("\nSaving results to CSV...")
df_perda = pd.DataFrame(resultado_perda, columns=['patch_id', 'neighborhood'])
df_ganho = pd.DataFrame(resultado_ganho, columns=['patch_id', 'neighborhood'])

# Check if the output file name is correct (was 1985_1986, now 1986_1987)
df_perda.to_csv(os.path.join(base_dir, 'resultado_neighborhood_loss_2013_2018.csv'), index=False)
df_ganho.to_csv(os.path.join(base_dir, 'resultado_neighborhood_gain_2013_2018.csv'), index=False)

print("Results saved successfully.")

In [4]:
!pip install rasterio

In [ ]:
import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Compression
from rasterio.errors import RasterioIOError #IMPORT TO HANDLE I/O ERRORS

# Base directory on Google Drive 
base_dir = r'E:\Process'

# Raster parameters
# Consistency in variable names (camelCase to snake_case, by Python convention)
COLS = 12989
ROWS = 9278
CELL_SIZE_X = 0.0002694945852358565
CELL_SIZE_Y = 0.00026949458523585663
UPPER_LEFT_X = -55.500254
UPPER_LEFT_Y = -16.499806

# Geographic transformation
transform = from_origin(UPPER_LEFT_X, UPPER_LEFT_Y, CELL_SIZE_X, CELL_SIZE_Y)

# Coordinate reference system WGS 84 (EPSG:4326)
CRS = 'EPSG:4326'

# Optimized function to convert neighborhood CSV to GeoTIFF using patch reference
def neighborhood_to_geotiff_optimized(csv_path, npy_patch_path, output_tif_path):
    print(f"Starting conversion: {os.path.basename(csv_path)}")

    try:
        # Load CSV: usecols to load only necessary columns, reducing memory consumption
        df = pd.read_csv(csv_path, usecols=['patch_id', 'neighborhood'])
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_path}")
        return
    except pd.errors.EmptyDataError:
        print(f"Error: CSV file is empty or malformed at {csv_path}")
        return

    try:
        # Load patches
        patches = np.load(npy_patch_path)
    except FileNotFoundError:
        print(f"Error: NPY file not found at {npy_patch_path}")
        return
    except Exception as e:
        print(f"Error loading NPY file {npy_patch_path}: {e}")
        return

    # Ensure that the dimensions of patches match the expected ones
    if patches.shape != (ROWS, COLS):
        print(f"Warning: Dimensions of NPY file ({patches.shape}) do not match the expected ({ROWS}, {COLS}).")
        # You may choose to resize or raise an error depending on desired behavior
        # Ex: patches = np.resize(patches, (ROWS, COLS))

    # Initialize the output raster with zeros, using the same dtype as neighborhood values or np.uint8
    # If 'neighborhood' can have values outside uint8, adjust the dtype (e.g., np.int16)
    # It is crucial that raster_out has the same data type as the output GeoTIFF for efficiency
    raster_out = np.zeros_like(patches, dtype=np.uint8) # Assuming 'neighborhood' fits in uint8

    # Main optimization: Vectorized mapping
    # 1. Create an auxiliary array to map patch_id to neighborhood
    # Determine the maximum patch_id value to size the mapping array
    max_patch_id = df['patch_id'].max()
    if max_patch_id >= (2**32 - 1): # If patch_id is very large, using a dictionary might be better
        print("Warning: patch_id is very large, using dictionary for mapping.")
        patch_id_to_neighborhood = dict(zip(df['patch_id'], df['neighborhood']))
        # Iterate over patches and apply mapping
        for i in np.unique(patches): # Iterate over unique patch IDs present in the raster
            if i in patch_id_to_neighborhood:
                raster_out[patches == i] = patch_id_to_neighborhood[i]
    else:
        # Create a lookup array where the index is the patch_id and the value is the neighborhood
        # Initialize with 0 (or an appropriate 'no data' value)
        lookup_table = np.zeros(max_patch_id + 1, dtype=np.uint8)
        # Fill the lookup table with neighborhood values
        lookup_table[df['patch_id'].values] = df['neighborhood'].values

        # Apply the vectorized mapping: for each pixel in 'patches', use its value as an index into the lookup_table
        # This is SIGNIFICANTLY faster than the original loop.
        raster_out = lookup_table[patches]

    print("Saving GeoTIFF...")
    try:
        with rasterio.open(
            output_tif_path,
            'w',
            driver='GTiff',
            height=ROWS,
            width=COLS,
            count=1,
            dtype=raster_out.dtype, # Use the dtype of the output array
            crs=CRS,
            transform=transform,
            compress=Compression.lzw, # LZW compression is lossless and efficient
            tiled=True,              # Enable tiling for better read/write performance on large files
            blockxsize=256,          # Block size for tiling (adjust if necessary)
            blockysize=256           # Block size for tiling (adjust if necessary)
        ) as dst:
            dst.write(raster_out, 1)
        print(f"GeoTIFF saved: {output_tif_path}\n")
    except RasterioIOError as e:
        print(f"I/O error while saving GeoTIFF {output_tif_path}: {e}")
    except Exception as e:
        print(f"Unexpected error while saving GeoTIFF {output_tif_path}: {e}")
# File paths 
csv_perda = os.path.join(base_dir, 'resultado_vizinhanca_perda_2013_2018.csv')
csv_ganho = os.path.join(base_dir, 'resultado_vizinhanca_ganho_2013_2018.csv')
npy_perda = os.path.join(base_dir, 'perda_patches_2013_2018.npy')
npy_ganho = os.path.join(base_dir, 'ganho_patches_2013_2018.npy')
tif_perda = os.path.join(base_dir, 'vizinhanca_perda_2013_2018.tif')
tif_ganho = os.path.join(base_dir, 'vizinhanca_ganho_2013_2018.tif')

# Execute conversion with the optimized function
vizinhanca_para_geotiff_otimizado(csv_perda, npy_perda, tif_perda)
vizinhanca_para_geotiff_otimizado(csv_ganho, npy_ganho, tif_ganho)

Iniciando conversão: resultado_vizinhanca_perda_2015_2016.csv
Salvando GeoTIFF...
GeoTIFF salvo: E:\Process\vizinhanca_perda_2015_2016.tif

Iniciando conversão: resultado_vizinhanca_ganho_2015_2016.csv
Salvando GeoTIFF...
GeoTIFF salvo: E:\Process\vizinhanca_ganho_2015_2016.tif

